In [46]:
import pandas as pd

In [47]:
import time

In [71]:
import datetime as dt

In [97]:
import pytz
eastern = pytz.timezone('US/Eastern')

In [48]:
pd.set_option('display.max_columns', None)

In [105]:
def clean_file(df):
    df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)
    for i in range(len(df)):
        start = df.at[i, 'Published Start']
        if start[-1] == 'a' or start[0:2] == '12':
            if len(start.split(':')[0]) == 1:
                df.at[i, 'Published Start'] = '0%s' %(start[:-1])
            else:
                df.at[i, 'Published Start'] = start[:-1]
        elif start == 'noon':
            df.at[i, 'Published Start'] = '12:00'
        else:
            hour = int(start.split(':')[0])
            hour += 12
            rest = start.split(':')[1]
            df.at[i, 'Published Start'] = '%d:%s' %(hour,rest[0:2])


        end = df.at[i, 'Published End']
        if end[-1] == 'a' or end[0:2] == '12':
            if len(end.split(':')[0]) == 1:
                df.at[i, 'Published End'] = '0%s' %(end[:-1])
            else:
                df.at[i, 'Published End'] = end[:-1]
        elif end == 'noon':
            df.at[i, 'Published End'] = '12:00'
        else:
            hour = int(end.split(':')[0])
            hour += 12
            rest = end.split(':')[1]
            df.at[i, 'Published End'] = '%d:%s' %(hour,rest[0:2])

        df.at[i, 'Building'] = df.at[i, 'Location'].split()[0]
        df.at[i, 'Room'] = df.at[i, 'Location'].split()[1]

def combine_time(df):
    for i in range(len(df)):
        df.at[i, "Start Time"] = df.at[i, "Date"] + "T" + df.at[i, "Published Start"]
        df.at[i, "End Time"] = df.at[i, "Date"] + "T" + df.at[i, "Published End"]

def convert_iso(df):
    df["Start Time"] = pd.to_datetime(df["Start Time"])
    df["End Time"] = pd.to_datetime(df["End Time"])

def to_epoch(df):
    for i in range(len(df)):
        start = eastern.localize(df.at[i, "Start Time"])
        end = eastern.localize(df.at[i, "End Time"])
        df.at[i, "Start Time"] = start.timestamp()
        df.at[i, "End Time"] = end.timestamp()

def drop_useless_cols(df):
    df = df[['Name', 'Section', 'Start Time', 'End Time', 'Building', 'Room', 'Capacity', 'Address', 'Lat', 'Long']]

def insert_address(df, address):
    for i in range(len(address)):
        arr = address.at[i, "Combined"].split(', ')
        address.at[i, "Lat"] = arr[0]
        address.at[i, "Long"] = arr[1]
    address = address.drop(["Combined"], axis=1)
    df = pd.merge(df, address, on="Building")
    return df

def insert_beg_end(df):
    for date in set(df["Date"]):
        for classroom in set(df["Location"]):
            df.loc[len(df)] = [pd.NA, pd.NA, pd.NA, pd.NA, pd.NA, date, '06:00', '6:00', classroom, df[df.Location == classroom].iloc[0]['Capacity'], pd.NA, pd.NA, pd.NA, pd.NA, df[df.Location == classroom].iloc[0]['Building'], df[df.Location == classroom].iloc[0]['Room'], df[df.Location == classroom].iloc[0]['Address'], df[df.Location == classroom].iloc[0]['Lat'], df[df.Location == classroom].iloc[0]['Long']]
            df.loc[len(df)] = [pd.NA, pd.NA, pd.NA, pd.NA, pd.NA, date, '22:00', '22:00', classroom, df[df.Location == classroom].iloc[0]['Capacity'], pd.NA, pd.NA, pd.NA, pd.NA, df[df.Location == classroom].iloc[0]['Building'], df[df.Location == classroom].iloc[0]['Room'], df[df.Location == classroom].iloc[0]['Address'], df[df.Location == classroom].iloc[0]['Lat'], df[df.Location == classroom].iloc[0]['Long']]

In [106]:
df = pd.read_csv("meetings.csv")

In [107]:
address = pd.read_csv("combined_address.csv")

In [108]:
clean_file(df)

In [109]:
df = insert_address(df, address)

In [54]:
insert_beg_end(df)

In [55]:
df_copy = df.copy()

In [56]:
df = df_copy.copy()

In [110]:
combine_time(df)

In [111]:
convert_iso(df)

In [89]:
drop_useless_cols(df)

In [112]:
to_epoch(df)

In [61]:
df = df.drop(['Name', 'Section', 'Type', 'Title', 'Note', 'Approved', 'Instructor / Organization',
       'Email', 'Requested Services'], axis=1)

In [62]:
sorted_df = df.sort_values("Published Start")

In [63]:
date_list = []
for d_index, date in enumerate(sorted(list(set(df["Date"])))):
    class_list = list()
    for c_index, classroom in enumerate(sorted(list(set(sorted_df["Location"])))):
        temp_df = sorted_df[(sorted_df["Location"] == classroom) & (sorted_df["Date"] == date)].reset_index(drop=True)
        if len(temp_df) == 0:
            continue
        class_list.append(temp_df)
    date_list.append(class_list)

In [64]:
# things = list()

In [65]:
# thing = pd.DataFrame(things, columns=['Date', 'Published Start', 'Published End', 'Location', 'Capacity',
#        'Building', 'Room', 'Address', 'Lat', 'Long', 'Start Time',
#        'End Time'])

In [66]:
# day = 0
# room = 0
# temp_things = list()
# temp_df = date_list[0][0].values
# for i in range(1, len(date_list[day][room])):
#     print(i)
#     temp_row = temp_df[0].copy()
#     temp_row[10] = int(temp_df[i - 1][11])
#     temp_row[11] = int(temp_df[i][10])
#     temp_start = time.gmtime(temp_row[10])
#     temp_end = time.gmtime(temp_row[11])
#     temp_row[1] = '{0:02d}:{1:02d}'.format(temp_start[3], temp_start[4])
#     temp_row[2] = '{0:02d}:{1:02d}'.format(temp_end[3], temp_end[4])
#     temp_things.append(temp_row)

# temp_things = pd.DataFrame(temp_things, columns=['Date', 'Published Start', 'Published End', 'Location', 'Capacity',
#        'Building', 'Room', 'Address', 'Lat', 'Long', 'Start Time',
#        'End Time'])

In [67]:
final = list()
for day in range(len(date_list)):
    day_list = list()
    for room in range(len(date_list[day])):
        room_list = list()
        temp_df = date_list[day][room].values
        for i in range(1, len(date_list[day][room])):
            temp_row = temp_df[0].copy()
            temp_row[10] = int(temp_df[i - 1][11])
            temp_row[11] = int(temp_df[i][10])
            temp_start = time.gmtime(temp_row[10])
            temp_end = time.gmtime(temp_row[11])
            temp_row[1] = '{0:02d}:{1:02d}'.format(temp_start[3], temp_start[4])
            temp_row[2] = '{0:02d}:{1:02d}'.format(temp_end[3], temp_end[4])
            room_list.append(temp_row)
        thing = pd.DataFrame(room_list, columns=['Date', 'Published Start', 'Published End', 'Location', 'Capacity', 'Building', 'Room', 'Address', 'Lat', 'Long', 'Start Time','End Time'])
        day_list.append(thing)
    final.append(day_list)      

In [68]:
final_df = pd.DataFrame()
for i in range(len(final)):
    temp_concat = pd.concat(final[i])
    final_df = pd.concat([final_df, temp_concat])

In [69]:
final_df.tail()

,Date,Published Start,Published End,Location,Capacity,Building,Room,Address,Lat,Long,Start Time,End Time
4,03/31/2023,12:20,12:30,WTHR 420,49,WTHR,420,560 Oval Drive,40.42666922643026,-86.91307453044112,1680265200,1680265800
5,03/31/2023,13:20,13:30,WTHR 420,49,WTHR,420,560 Oval Drive,40.42666922643026,-86.91307453044112,1680268800,1680269400
6,03/31/2023,14:20,14:30,WTHR 420,49,WTHR,420,560 Oval Drive,40.42666922643026,-86.91307453044112,1680272400,1680273000
7,03/31/2023,15:20,17:30,WTHR 420,49,WTHR,420,560 Oval Drive,40.42666922643026,-86.91307453044112,1680276000,1680283800
8,03/31/2023,20:00,22:00,WTHR 420,49,WTHR,420,560 Oval Drive,40.42666922643026,-86.91307453044112,1680292800,1680300000


In [70]:
final_df.to_csv('sprint2.csv', index=False, header=False)